In [1]:
!pip install gdown --no-cache-dir -U

## Import libraries

In [77]:
import os
import re
import random
import time
from timeit import default_timer as timer
from typing import Iterable, List
from tqdm import tqdm
import h5py
import pathlib as pl
from pathlib import Path
import pandas as pd
import numpy as np
from d2l import torch as d2l
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchtext.data import get_tokenizer
from torch.nn.utils.rnn import pad_sequence
from torchtext.vocab import GloVe, vocab
from torchinfo import summary
%matplotlib inline

## Obtain the data

Get the data from google drive and extract it into the correct directory

In [ ]:
!gdown "18d7-qbKjt2uS1ORdvVIr8LBrTqdZYaTI"
!tar xvjf "./data/C4_200M.hdf5-00001.3-of-00010.tar.bz2"

## Dataset Object

Create an HDF5 dataset object which will be fed into the DataLoader later

In [2]:
class Hdf5Dataset(Dataset):
    """Custom Dataset for loading entries from HDF5 databases"""

    def __init__(self, h5_path: str, transform = None, num_entries: int = None):
        """Initialization function. Obtains the raw data as bytes, num entries, and transform, if any
        
        :param h5_path: a string representing a valid path to the hdf5 file to read
        :param transform: a function that is used to process the raw input (optional)
        :param num_entries: number of entries in the dataset (optional)
        :returns: an object of the Hdf5Datset
        """
        self.h5f = h5py.File(h5_path, "r") # open HDF5 file for reading
        if num_entries:
            self.num_entries = num_entries
        else: # if num_entries not given, set len of dataset to the len of the lables in the hdf5 file
            self.num_entries = self.h5f["labels"].shape[0]
        self.transform = transform

    def __getitem__(self, index):
        """obtains both the input as well as the labels at the given index. index may
           be an integer or slice
        
        :param index: integer or slice to index dataset with
        :returns: a tuple containing either a single input,label pair or list of inputs, list of labes pair
        """
        if isinstance(index, slice):
            input = [entry.decode("utf-8") for entry in self.h5f["input"][index]]
            label = [entry.decode("utf-8") for entry in self.h5f["labels"][index]]
        elif isinstance(index, int):
            if index > self.num_entries:
                raise StopIteration
            input = self.h5f["input"][index].decode("utf-8")
            label = self.h5f["labels"][index].decode("utf-8")
        # if self.transform is not None:
        #     features = self.transform(input)
        return input, label

    def __len__(self):
        """gets number of entries"""
        return self.num_entries

In [4]:
# playing with hdf5 data importing in python
root_path = "./"
data_path = f"{root_path}data/"
c4_hdf5_train_filename = "C4_200M.hdf5-00000-of-00010"
c4_hdf5_train_filepath = f"{data_path}{c4_hdf5_train_filename}"
c4_dataset = Hdf5Dataset(c4_hdf5_train_filepath) # no transforms yet, no len given

In [5]:
len(c4_dataset)
c4_dataset[-1]

('Preheat oven to 356 degrees Fahrenheit. (180 & degrees celsius), I don’t use a macro.wave so I put the coconut oil in the oven to melt it while preheated. (lifehack :-)) Some extra coconut oil to green coconut oil to grease your pan,',
 'Preheat oven to 356 degrees Fahrenheit (180 degrees celsius). I don’t have a microwave so I put the coconut oil in the oven to melt it while it is preheating (lifehack :-)). Use some extra coconut oil to grease your pan.')

## Global Variables

includes input parameters, vocabulary parameters, filepaths

In [6]:
# def yield_tokens(data_iter: Iterable, index: int, src_lang_lbl: str, tgt_lang_lbl: str) -> List[str]:
#     """Helper function to yield a list of tokens from a data iterable"""
#     language_index = {src_lang_lbl: 0, tgt_lang_lbl: 1}
#     for data_sample in tqdm(data_iter):
#         if data_sample[index] and isinstance(data_sample[index], str):
#             yield token_transform(data_sample[index])

SRC_LANGUAGE = "incorrect"
TGT_LANGUAGE = "correct"

MAX_LENGTH = 512 # maximum length of an input sequence
VOCAB_SIZE = 20000 # size of the vocabulary, something to play around with
N_TRAIN_SAMPLES = 1000000 # number of training samples to pull from the dataset
N_VAL_SAMPLES = int(0.1 * N_TRAIN_SAMPLES) # number of validation/test samples to pull from the dataset

# function-name placeholders for transforms
token_transform = get_tokenizer("basic_english")
vocab_transform = None

folder = "./data"
train_filename = "C4_200M.hdf5-00000-of-00010"
valid_filename = "C4_200M.hdf5-00001-of-00010"

embedding_path = "./glove.42B.300d.txt"

checkpoint_folder = "./checkpoints"

In [ ]:
import gdown

gdown.download_folder(
    "https://drive.google.com/drive/folders/1FQ_jm765fgwcD5lLtjl6ef9k532hdADR",
    quiet=True,
)

# Vocabulary

Create special tokens then create pre-trained GloVe vocabulary

In [7]:
# Define special symbols and indices
UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3

# Make sure the tokens are in order of their indices to properly insert them in vocab
special_symbols = ["<UNK>", "<PAD>", "<BOS>", "<EOS>"]

In [8]:
def pretrained_embs(name: str, dim: str, max_vectors: int = None):
    """get the pre-trained word embeddings for the vocabulary
    
    :param name: the name of the embedding, according to torchtext.vocab.GloVe
    :param dim: the desired dimensionality of the embedding
    :param max_vectors: maximum number of pre-trained vectors to load
    :returns: a tuple containing the str-to-int vocab dict and the embedded vectors of the vocab
    """
    glove_vectors = GloVe(name=name, dim=dim, max_vectors=max_vectors)
    glove_vocab = vocab(glove_vectors.stoi)
    pretrained_embeddings = glove_vectors.vectors
    glove_vocab.insert_token("<UNK>", UNK_IDX)
    pretrained_embeddings = torch.cat(
        (torch.mean(pretrained_embeddings, dim=0, keepdims=True), pretrained_embeddings)
    )
    glove_vocab.insert_token("<PAD>", PAD_IDX)
    pretrained_embeddings = torch.cat(
        (torch.zeros(1, pretrained_embeddings.shape[1]), pretrained_embeddings)
    )
    glove_vocab.insert_token("<BOS>", PAD_IDX)
    pretrained_embeddings = torch.cat(
        (torch.rand(1, pretrained_embeddings.shape[1]), pretrained_embeddings)
    )
    glove_vocab.insert_token("<EOS>", PAD_IDX)
    pretrained_embeddings = torch.cat(
        (torch.rand(1, pretrained_embeddings.shape[1]), pretrained_embeddings)
    )
    glove_vocab.set_default_index(UNK_IDX)
    return glove_vocab, pretrained_embeddings

In [72]:
vocab, embeddings = pretrained_embs("42B", "300", 20000)
print(type(vocab), type(embeddings))
torch.save(embeddings, "glove.42B.300d.20K.pth")

.vector_cache/glove.42B.300d.zip: 1.88GB [05:53, 5.32MB/s]                                                                                                                                                                                                                    
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉| 19999/20000 [00:01<00:00, 14166.85it/s]


<class 'torchtext.vocab.vocab.Vocab'> <class 'torch.Tensor'>


In [9]:
# Load vocabulary and pretrained embeddings
vocab_transform = torch.load("vocab/vocab_20K.pth")
embeddings = torch.load("glove.42B.300d.20K.pth")

---

## Collation

In [10]:
def sequential_transforms(*transforms):
    """Helper function to compose sequential operations
    
    param transforms: list of transformation functions to be applied
    :returns: the full transformation after composition of sequential ops
    """
    def func(txt_input):
        for transform in transforms:
            txt_input = transform(txt_input)
        return txt_input
    return func


def tensor_transform(token_ids: List[int]):
    """prepends and appends the given list of token ids with BOS id and EOS
    
    param token_ids: a list of integers representing the ids of the tokens in a sequence
    :returns: a pytorch tensor representation of the concatenated BOS id, ids list, and EOS id
    """
    return torch.cat(
        (torch.tensor([BOS_IDX]), torch.tensor(token_ids), torch.tensor([EOS_IDX]))
    )

# src and tgt language text transforms to convert raw strings into tensors indices
# with BOS and EOS prepended and appended resp.
text_transform = sequential_transforms(
    token_transform, vocab_transform, tensor_transform
)

# function to collate data samples into batch tesors
def collate_fn(batch):
    src_batch, tgt_batch = [], []
    for src_sample, tgt_sample in batch:
        src_batch.append(text_transform(src_sample.rstrip("\n")))
        tgt_batch.append(text_transform(tgt_sample.rstrip("\n")))

    src_batch = pad_sequence(src_batch, padding_value=PAD_IDX)
    tgt_batch = pad_sequence(tgt_batch, padding_value=PAD_IDX)
    return src_batch, tgt_batch

In [11]:
text = """According to all known laws of aviation, there
          is no way a bee should be able to fly."""
tokenized_input = token_transform(text)
print("tokenized input:\n", tokenized_input)

encoded_input = vocab_transform(tokenized_input)
print("encoded input:\n", encoded_input)

print("transformed input:\n", text_transform(text))

tokenized input:
 ['according', 'to', 'all', 'known', 'laws', 'of', 'aviation', ',', 'there', 'is', 'no', 'way', 'a', 'bee', 'should', 'be', 'able', 'to', 'fly', '.']
encoded input:
 [306, 8, 42, 529, 1888, 9, 4717, 5, 70, 13, 81, 138, 10, 6663, 129, 28, 315, 8, 2702, 4]
transformed input:
 tensor([   2,  306,    8,   42,  529, 1888,    9, 4717,    5,   70,   13,   81,
         138,   10, 6663,  129,   28,  315,    8, 2702,    4,    3])


## Unknown words

In [12]:
text = "lmk where ur at"
tokenized_input = token_transform(text)
print(tokenized_input)

encoded_input = vocab_transform(tokenized_input)
print(encoded_input)

print("transformed input:\n", text_transform(text))

['lmk', 'where', 'ur', 'at']
[0, 116, 11025, 23]
transformed input:
 tensor([    2,     0,   116, 11025,    23,     3])


## RNN Network

### Masking

We will definitely need a padding mask, and if we use self-attention (maybe later) we will also need a subsequent mask

In [13]:
def generate_square_subsequent_mask(sz, device='cpu'):
    """Create a mask that ignores future inputs

    :param sz: the size the input sequence to which the mask will be applied
    :returns: a tensor with shape (sz, sz)
    """
    mask = (torch.triu(torch.ones((sz, sz), device=device)) == 1).transpose(0, 1)
    mask = (
        mask.float()
        .masked_fill(mask == 0, float("-inf"))
        .masked_fill(mask == 1, float(0.0))
    )
    return mask


def create_padding_mask(src):
    """creates a padding mask which ignores PAD tokens when applied to an input

    :param src: the input sequence to compute the mask for
    :returns: a tensor with shape (src_seq_len, src_seq_len)
    """
    src_seq_len = src.shape[0]
    src_padding_mask = (src == PAD_IDX).transpose(0, 1)
    return src_padding_mask

In [14]:
example_size = 10
subseq_mask = generate_square_subsequent_mask(example_size)
subseq_mask

tensor([[0., -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., 0., -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., -inf],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [15]:
example_text = "lmk where ur at"
example_text_id_seq = text_transform(example_text.rstrip("\n"))

example_max_len = 50
num_pad = example_max_len - len(example_text_id_seq)
padded_example_text_id_seq = torch.cat((example_text_id_seq, torch.tensor([PAD_IDX] * num_pad)))
padded_example_text_id_seq = padded_example_text_id_seq.view(-1,1) # turn the tensor into a column vec
example_padding_mask = create_padding_mask(padded_example_text_id_seq)
print("example text ids before padding:\n", example_text_id_seq)
print()
print("example text ids after padding:\n", padded_example_text_id_seq.T)
print()
print("example text padding mask:\n", example_padding_mask)

example text ids before padding:
 tensor([    2,     0,   116, 11025,    23,     3])

example text ids after padding:
 tensor([[    2,     0,   116, 11025,    23,     3,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1]])

example text padding mask:
 tensor([[False, False, False, False, False, False,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True]])


### Model-Specific Parameters

In [16]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(DEVICE)

teacher_forcing_ratio = 0.5
torch.manual_seed(0)

MAX_LENGTH = 512
EMB_SIZE = 300
HIDDEN_SIZE = 512
BATCH_SIZE = 16
NUM_ENCODER_LAYERS = 1
NUM_DECODER_LAYERS = 1

learning_rate = 0.001

cuda


### The Model

The RNN we employ contains encoder block(s) and a decoder block(s) which themselves are comprised of GRU units.\
We Apply an embedding layer before the encoder/decoder blocks, and then we apply a single dropout layer for the\
encoding layer, and a fully connected output layer, followed by drop-out and softmax in the decoding layer.

#### TODO:

- check that the embeddings are not trainable
- double-check the architecture of the encoder
- double check the architecture of the decoder, particularly the fc outputs

In [80]:
def init_seq2seq(module):
    """Initialize weights for seq2seq"""
    if type(module) == nn.Linear:
        nn.init.xavier_uniform_(module.weight)
    if type(module) == nn.GRU:
        for param in module._flat_weights_names:
            if "weight" in param:
                nn.init.xavier_uniform_(module._parameters[param])

class Encoder(nn.Module):
    def __init__(self, vocab_sz, emb_dim, hid_dim, num_layers, dropout=0, embedding_weights=None):
        super().__init__()
        self.hid_dim = hid_dim
        self.embedding = nn.Embedding(vocab_sz, emb_dim, padding_idx=1)
        if embedding_weights is not None:
            self.embedding.weight = torch.nn.Parameter(
                torch.from_numpy(embedding_weights)
            )
        self.rnn = nn.GRU(emb_dim, hid_dim, num_layers=num_layers, batch_first=False, dropout=dropout)
        self.apply(init_seq2seq)

    def forward(self, src):
        embedded = self.embedding(src.t().type(torch.int64))
        outputs, hidden_state = self.rnn(embedded)  # no cell state!
        # outputs = [src len, batch size, hid dim * n directions]
        # hidden = [n layers * n directions, batch size, hid dim]
        # outputs are always from the top hidden layer
        return outputs, hidden_state

class Decoder(nn.Module):
    def __init__(self, vocab_sz, emb_dim, hid_dim, num_layers, dropout=0, embedding_weights=None):
        super().__init__()
        self.hid_dim = hid_dim
        self.output_dim = vocab_sz
        self.embedding = nn.Embedding(vocab_sz, emb_dim, padding_idx=1)
        if embedding_weights is not None:
            self.embedding.weight = torch.nn.Parameter(
                torch.from_numpy(embedding_weights)
            )
        self.rnn = nn.GRU(emb_dim + hid_dim, hid_dim, num_layers=num_layers, dropout=dropout)
        self.fc_out = nn.LazyLinear(vocab_size)
        #self.softmax = nn.LogSoftmax(dim=1)

    def init_state(self, enc_all_outputs, *args):
        return enc_all_outputs
    
    def forward(self, input, state):
        # input shape: (batch_size, num_t_steps)
        # embs shape: (num_t_steps, batch_size, embed_size)
        embs = self.embedding(input.t().type(torch.int64))
        enc_output, hidden_state = state
        # context shape: (batch_size, num_hiddens) (we grab the last output)
        context = enc_output[-1]
        # broadcast context to (num_t_steps, batch_size, num_hiddens)
        context = context.repeat(embs.shape[0], 1, 1)
        # concat along feature dimension
        print(embs.shape)
        print(context.shape)
        embs_and_context = torch.cat((embs, context), -1)
        outputs, hidden_state = self.rnn(embs_and_context, hidden_state)
        outputs = self.fc_out(outputs).swapaxes(0, 1) #-> why swap?
        return outputs, [enc_output, hidden_state]

In [56]:
# testing out the encoder and decoder
vocab_size, embed_size, num_hiddens, num_layers = 10, 8, 16, 2
batch_size, num_t_steps = 4, 9
encoder = Encoder(vocab_size, embed_size, num_hiddens, num_layers)
X = torch.zeros((batch_size, num_steps))
enc_outputs, enc_state = encoder(X)
d2l.check_shape(enc_outputs, (num_t_steps, batch_size, num_hiddens))
d2l.check_shape(enc_state, (num_layers, batch_size, num_hiddens))

decoder = Decoder(vocab_size, embed_size, num_hiddens, num_layers)
dec_outputs, state = decoder(X, (enc_outputs, enc_state))
d2l.check_shape(dec_outputs, (batch_size, num_t_steps, vocab_size))
d2l.check_shape(state[1], (num_layers, batch_size, num_hiddens))

In [57]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        assert (
            encoder.hid_dim == decoder.hid_dim
        ), "Hidden dimensions of encoder and decoder must be equal!"

    def forward(self, src, trg, teacher_forcing=0.5):
        # enc_all_outputs = self.encoder(src, *args)
        # dec_state = self.decoder.init_state(enc_all_outputs, *args)
        # return self.decoder(trg, dec_state)[0]

        # teacher_forcing_ratio is probability to use teacher forcing
        # e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        # tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        # last hidden state of the encoder is the context
        enc_outputs, state = self.encoder(src)
        # first input to the decoder is the <bos> tokens
        input = trg[0, :]
        for t in range(1, trg_len):
            # insert input token embedding, previous hidden state and the context state
            # receive output tensor (predictions) and new hidden state
            dec_outputs, state = self.decoder(input, (enc_outputs, state))
            # place predictions in a tensor holding predictions for each token
            outputs[t] = dec_outputs
            # if teacher forcing, use actual next token as next input
            # if not, use highest predicted token from predictions
            input = trg[t] if np.random.rand() < teacher_forcing else dec_outputs.argmax(-1)

        return outputs

In [58]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [84]:
# attn = Attention(HIDDEN_SIZE, HIDDEN_SIZE)

encoder1 = Encoder(
    VOCAB_SIZE, EMB_SIZE, HIDDEN_SIZE, NUM_ENCODER_LAYERS, dropout=0, embedding_weights=np.array(embeddings)
)
decoder1 = Decoder(
    VOCAB_SIZE, EMB_SIZE, HIDDEN_SIZE, NUM_DECODER_LAYERS, dropout=0.1, embedding_weights=np.array(embeddings)
)

loss_fn = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX)

# optimizer = torch.optim.Adam(encoder1.parameters(), lr = learning_rate , betas=(0.9, 0.98), eps=1e-9)
# decoder_optimizer = torch.optim.Adam(encoder1.parameters(), lr = learning_rate, betas=(0.9, 0.98), eps=1e-9)

model = Seq2Seq(encoder1, decoder1, DEVICE)
#model = nn.DataParallel(model, device_ids=[0,1,2,3])
model.to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, betas=(0.9, 0.98), eps=1e-9)

summary(model, input_size=[(BATCH_SIZE, MAX_LENGTH), (BATCH_SIZE, MAX_LENGTH)], device='cuda')
#print(f"The model has {count_parameters(model):,} trainable parameters")

torch.Size([512, 300])
torch.Size([512, 16, 512])


RuntimeError: Failed to run torchinfo. See above stack traces for more details. Executed layers up to: [Encoder: 1, Embedding: 2, GRU: 2, Embedding: 2]

In [ ]:
def train(model, iterator, optimizer, loss_fn, clip):
    model.train() # set model into training mode so apply dropout, etc.
    epoch_loss = 0
    for src, trg in tqdm(iterator):
        optimizer.zero_grad()
        src = src.to(DEVICE)
        trg = trg.to(DEVICE)
        output = model(src, trg)
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        output_dim = output.shape[-1]
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        loss = loss_fn(output, trg)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        epoch_loss += loss.item()
    return epoch_loss / len(iterator)

def evaluate(model, iterator, criterion):
    model.eval() # set model into inference mode, so DONT apply dropout
    epoch_loss = 0
    with torch.no_grad():
        for src, trg in tqdm(iterator):
            src = src.to(DEVICE)
            trg = trg.to(DEVICE)
            output = model(src, trg, 0) #turn off teacher forcing
            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]
            output_dim = output.shape[-1]
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)
            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size]
            loss = criterion(output, trg)
            epoch_loss += loss.item()
    return epoch_loss / len(iterator)


In [16]:
current_time = lambda: time.strftime("%Y-%m-%d-%H:%M:%S", time.localtime())
print(current_time())

2023-07-28-07:01:45


In [ ]:
NUM_EPOCHS = 1
CLIP = 1 # gradient clipping
RESUME = False

train_iter = Hdf5Dataset(
    pl.Path(folder) / train_filename, num_entries=N_TRAIN_SAMPLES)
train_dataloader = DataLoader(
    train_iter, batch_size=BATCH_SIZE, collate_fn=collate_fn)
val_iter = Hdf5Dataset(pl.Path(folder) / valid_filename, num_entries=N_VAL_SAMPLES)
val_dataloader = DataLoader(
    val_iter, batch_size=BATCH_SIZE, collate_fn=collate_fn)

# make sure folder exists
pl.Path("checkpoints").mkdir(parents=True, exist_ok=True)

model.train()
if RESUME:
    checkpoint = torch.load(
        pl.Path("checkpoints") /
        f"model-epoch_{NUM_EPOCHS-1}-{current_time()}.pt"
    )
    model.load_state_dict(checkpoint["model_state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
    epoch = checkpoint["epoch"]

for epoch in range(1, NUM_EPOCHS + 1):
    start_time = timer()
    print(
        f"\033[92mEpoch {epoch} of {NUM_EPOCHS} - time: {current_time()}\033[0m")
    print(f"\033[92mTraining...\033[0m")
    train_loss = train(model, train_dataloader, optimizer, loss_fn, CLIP)
    end_time = timer()
    print(f"\033[92mValidating...\033[0m")
    val_loss = evaluate(model, val_dataloader, loss_fn)
    print(
        (
            f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}, "
            f"Epoch time = {(end_time - start_time):.3f}s"
        )
    )
    torch.save(
        {
            "epoch": epoch,
            "model_state_dict": model.state_dict(),
            "optimizer_state_dict": optimizer.state_dict(),
            "loss": val_loss,
        },
        pl.Path("checkpoints") /
        f"model-epoch_{NUM_EPOCHS-1}-{current_time()}.pt",
    )

Epoch 1 of 1 - time: 2023-07-27-23:50:47
Training...


 63%|█████████████████████████████████████████████████████████████████████████████████████████▋                                                    | 39489/62500 [2:42:08<1:24:09,  4.56it/s]

In [17]:
# function to generate output sequence using greedy algorithm
def correct_sentence_vectorized(src_tensor, model, max_len=50):
    assert isinstance(src_tensor, torch.Tensor)

    model.eval()
    src_tensor = src_tensor.unsqueeze(1).to(DEVICE)
    # get length of input sentence
    src_len = src_tensor.shape[0]

    trg_vocab_size = model.decoder.output_dim

    # tensor to store decoder outputs
    outputs = torch.zeros(max_len, 1, trg_vocab_size).to(DEVICE)

    # last hidden state of the encoder is the context
    with torch.no_grad():
        context = model.encoder(src_tensor)

    # context also used as the initial hidden state of the decoder
    hidden = context

    # first input to the decoder is the <sos> tokens
    input = src_tensor[0, :]
    # enc_src = [batch_sz, src_len, hid_dim]
    # Even though some examples might have been completed by producing a <eos> token
    # we still need to feed them through the model because other are not yet finished
    # and all examples act as a batch. Once every single sentence prediction encounters
    # <eos> token, then we can stop predicting.
    for t in range(1, max_len):
        # insert input token embedding, previous hidden state and the context state
        # receive output tensor (predictions) and new hidden state
        output, hidden = model.decoder(input, hidden, context)

        # place predictions in a tensor holding predictions for each token
        outputs[t] = output

        # get the highest predicted token from our predictions
        top1 = output.argmax(1)

        # if teacher forcing, use actual next token as next input
        # if not, use predicted token
        input = top1

    pred_sentence = []

    for i in range(1, len(outputs)):
        topv, topi = outputs[i, :, :].topk(1)
        pred_sentence.append(vocab_transform.vocab.itos_[topi])
        if topi == EOS_IDX:
            break

    return " ".join(pred_sentence)

In [19]:
latest_checkpoint = sorted(Path("checkpoints").glob("*.pt"), key=os.path.getmtime)[-1]
print(latest_checkpoint)
checkpoint = torch.load(latest_checkpoint)
model.load_state_dict(checkpoint["model_state_dict"])

model.eval()

# Pick one in 18M examples
val_iter = Hdf5Dataset(pl.Path(folder) / valid_filename, num_entries=None)

src, trg = np.random.choice(val_iter)

print('input: "', src, '"')
print('target: "', trg, '"')

src = text_transform(src)

print(f"\033[91mModel's prediction: \033[0m", end="")
print(correct_sentence_vectorized(src, model))

checkpoints/model-epoch_38-2023-07-27-23:29:53.pt


KeyboardInterrupt: 